## Imports

In [4]:
import random
from pprint import pprint
from pathlib import Path

import pandas as pd

from itertools import product
import itertools

from matplotlib import pyplot as plt


import Levenshtein as lev
from Levenshtein import distance as levdist
from Levenshtein import hamming as hamming

## Import own things

In [5]:
from tree import Tree
import partialdistancematrix
# import init_module    # init_module was a copy of __init__.py used for accessing dnc_tree

In [6]:
%load_ext autoreload
%autoreload 2

## Copy of `connect_the_trees`

Copied the function from `__init__.py` so that we can use it here.

In [159]:
def connect_the_trees(subtrees, connecting_node):
    '''
    Merge the subtrees. They are sharing a unique vertex, so it is guaranteed to connect the subtrees.
    '''
    first_tree = subtrees[0]
    for t in subtrees[1:]:
        first_tree.merge(t, connecting_node)
    first_tree.set_start_node(connecting_node)
    return first_tree

## Center vertex

In [7]:
def get_edge_weight(tree: Tree, edge_weights, edge):
    """
    edge=(x, y) is the directed edge from vertex x to y.
    Returns the length of the longest, simple path from a leaf to y such that the
    last edge in the path is (x, y), that is, reaching y via x.

    The function uses memoization to avoid recomputing the same edge weight
    multiple times. It makes recursive calls to itself. Computer edge weights
    are stored in the dictionary edge_weights.
    """
    if edge in edge_weights:
        return edge_weights[edge]
    x, y = edge
    neighbors = tree.adjacencies[x]
    if len(neighbors) == 1:  # x is a leaf.
        edge_weights[edge] = 1  # The edge (x,y) itself.
    else:
        #    n2   n1
        #      \ /
        # n3 -- x --- y
        #      / \
        #    n4   n5
        edge_weights[edge] = 1 + max(
            get_edge_weight(tree, edge_weights, (n, x)) for n in neighbors if n != y
        )
    return edge_weights[edge]

In [8]:
def get_weights(tree: Tree):
    """
    Returns two dictionaries: vertex_weights and edge_weights.
    vertex_weights[v] is the length of the longest, simple path from a leaf to v.
    edge_weights[(x,y)] is the length of the longest, simple path from a leaf to y
    such that the last edge in the path is (x, y).

    The edge weights are returned for visualization purposes and may be omitted later.
    """
    vertex_weights = {}
    edge_weights = {}
    for v, neighbors in tree.adjacencies.items():
        vertex_weights[v] = max(
            get_edge_weight(tree, edge_weights, (n, v)) for n in neighbors
        )
    return vertex_weights

In [9]:
def get_center_vertex(tree: Tree):
    """
    Returns the center vertex of the tree.
    """
    vertex_weights = get_weights(tree)
    return min(vertex_weights, key=vertex_weights.get)  # Break ties arbitrarily.

## Clades and path weights from center vertex

In [10]:
def get_path_weights_via_edge(tree: Tree, dm, edge):
    """
    For a given edge (x, y), the function returns a dictionary of path weights
    for leaves that can be reached when going from x via y.
    Example:
        Suppose that a leaf f can be reached from x via y (along a simple path).
        Suppose the sum of the edge weights on the simple path from x to f is 100.
        Then the returned dictionary will contain the key f with value 100.
    """
    x, y = edge
    xy_weight = dm.get(x, y)
    y_neighbors = tree.adjacencies[y]
    if len(y_neighbors) == 1:
        # y is a leaf.
        return {y: xy_weight}
    else:
        # y has other neighbors n1, n2 in addition to x.
        #        n1
        #       /
        # x -- y
        #       \
        #        n2
        path_weights = {}
        for n in y_neighbors:
            if n != x:
                for leaf, weight in get_path_weights_via_edge(tree, dm, (y, n)).items():
                    path_weights[leaf] = weight + xy_weight
        return path_weights

In [11]:
def get_clades_with_taxa_dists_to_center(tree: Tree, dm, split_vertex):
    """
    Returns a list of dictionaries, one for each edge leaving the split vertex.
    The dictionary associated with the edge e leaving the split vertex contains
    all leaves that can be reached from the split vertex via e. A leaf is mapped
    to the sum of the edge weights on the simple path from the split vertex to
    the leaf.

    In particular, if the split vertex is the center vertex of our constructed tree,
    there will be three dictionaries, each one representing a clade.
    """
    neighbors = tree.adjacencies[split_vertex]
    return [get_path_weights_via_edge(tree, dm, (split_vertex, n)) for n in neighbors]

## Neighbor joining

Different from the version in `__init__.py` as this one calculates the edge weights, adding them to the distance matrix.

In [13]:
def nj_selection_function(dm, current_leaves):
    def sum_distances(a):
        sum_for_a = 0
        for b in current_leaves:
            if a != b:
                sum_for_a += dm.get(a, b)
        return sum_for_a

    n_minus_2 = len(current_leaves) - 2
    Q_ij_min = n_minus_2 * dm.get(
        current_leaves[0], current_leaves[1]
    )  # Arbitrary but safe starting value that won't be chosen.
    best_so_far = None
    for x, y in itertools.combinations(current_leaves, 2):
        sum_distances_x = sum_distances(x)
        sum_distances_y = sum_distances(y)
        Q_ij = n_minus_2 * dm.get(x, y) - sum_distances_x - sum_distances_y
        if best_so_far is None or Q_ij < Q_ij_min:
            Q_ij_min = Q_ij
            best_so_far = (x, y, sum_distances_x, sum_distances_y)

    return best_so_far


def dnc_neighborjoining(dm, taxa, verbose=[]):
    """
    An implementation of NJ meant to compute base cases in dnctree.

    dm:       a PartialDistanceMatrix instance
    taxa:     a list of identifiers (part of dm) to infer the tree for
    verbose:  boolean, whether to print extra info to stderr
    """
    t = Tree()
    if len(taxa) == 1:
        t.add_vertex(taxa[0])
        return t
    elif len(taxa) == 2:
        t.add_edge(taxa[0], taxa[1])
        ######### Possibly add distance in dm for the single edge
        return t
    else:
        current_leaves = taxa[:]  # Slicing out a copy
        while len(current_leaves) > 3:
            x, y, sum_x_to_others, sum_y_to_others = nj_selection_function(
                dm, current_leaves
            )
            # if verbose:
            #     print(f'Joining {x} and {y}', file=sys.stderr)
            other_taxa = current_leaves[:]
            other_taxa.remove(x)
            other_taxa.remove(y)
            new_v = dm.create_representative_vertex(x, y, other_taxa)

            d_x_newv = 0.5 * dm.get(x, y) + (1 / (2 * (len(current_leaves) - 2))) * (
                sum_x_to_others - sum_y_to_others
            )
            d_y_newv = dm.get(x, y) - d_x_newv
            dm.set(x, new_v, d_x_newv)
            dm.set(y, new_v, d_y_newv)

            other_taxa.append(new_v)
            t.add_edge(x, new_v)
            t.add_edge(y, new_v)
            current_leaves = other_taxa

        c = dm.create_unique_vertex_id()
        for leaf in current_leaves:
            t.add_edge(leaf, c)
        t.set_start_node(c)
        x, y, z = current_leaves
        d_xc = 0.5 * dm.get(x, y) + (1 / 2) * (dm.get(x, z) - dm.get(y, z))
        d_yc = dm.get(x, y) - d_xc
        d_zc = dm.get(x, z) - d_xc
        dm._dm[c] = dict()
        dm.set(x, c, d_xc)
        dm.set(y, c, d_yc)
        dm.set(z, c, d_zc)
        return t

## Mock MSA

Generates random strings and the distance method returns the Hamming distance.

In [14]:
def random_string(length, alphabet):
    return "".join([random.choice(alphabet) for _ in range(length)])

In [15]:
class MockMSA:
    def __init__(self, n_taxa=10, taxa_length=10, alphabet="ab"):
        """
        Create (up to) n_taxa random strings of length taxa_length over specified alphabet.
        """
        taxa_set = set(random_string(taxa_length, alphabet) for _ in range(n_taxa))
        self.taxa_dict = {f"T_{i}": taxon for i, taxon in enumerate(taxa_set)}
        self._taxa = list(self.taxa_dict.keys())

    def can_retrieve_distances(self) -> bool:
        return True

    def distance(self, taxon1, taxon2):
        return float(hamming(self.taxa_dict[taxon1], self.taxa_dict[taxon2]))

    def taxa(self):
        return self._taxa

    def distance_matrix(self):
        """
        Not needed but for visualisation and debugging purposes.
        """
        df = pd.DataFrame(index=self.taxa(), columns=self.taxa())
        for taxon1, taxon2 in product(self.taxa(), self.taxa()):
            df.loc[taxon1, taxon2] = self.distance(taxon1, taxon2)
        return df

## Output nodes, edges and weights to be used with *csacademy* graph editor

In [16]:
def write_graph_data(t: Tree, dm: partialdistancematrix.PartialDistanceMatrix):
    """
    Write the tree with edge weights to a file. Each line is of the form:
    node1 node2 weight
    This data can be plotted with `https://csacademy.com/app/graph_editor/`
    """
    lines = []

    for node, edges in t.adjacencies.items():
        for neighbour in edges:
            # print(f"{node} {neighbour}")
            if node in dm._dm and neighbour in dm._dm[node]:
                weight = f"{dm._dm[node][neighbour]:.2f}"
            else:
                weight = ""
            lines.append(f"{node} {neighbour} {weight}".strip())

    print(f"Number of lines: {len(lines)}")
    text = "\n".join(lines)
    _ = (Path.home() / "temp" / "dnc_neighborjoining.txt").write_text(text)

## Try Neighbour-Joining

Only experimenting with the updated NJ algorithm above.

In [132]:
msa = MockMSA(n_taxa=20, taxa_length=10, alphabet="ab")
dm = partialdistancematrix.PartialDistanceMatrix(msa, "Distance function dummy")
t = dnc_neighborjoining(dm, msa.taxa())
write_graph_data(t, dm)
c = get_center_vertex(t)
print(f"Center vertex: {c}")
print(dm._dm[c])
clades = get_clades_and_path_weights(t, dm, c)
pprint(clades)
# display(msa.distance_matrix())

Number of lines: 74
Center vertex: #15
{'#6': 2.0, '#7': 1.15625, '#8': 1.109375, '#13': 0.34375, '#14': 0.328125, '#16': 0.578125, '#17': 0.390625}
[{'T_10': 3.832638888888889,
  'T_12': 3.210267857142857,
  'T_13': 4.054861111111111,
  'T_14': 3.81875,
  'T_18': 3.6650997899159665,
  'T_2': 4.530555555555555,
  'T_6': 3.576864495798319,
  'T_9': 4.419444444444444},
 {'T_19': 2.90625, 'T_5': 2.75},
 {'T_0': 3.3117897727272725,
  'T_1': 4.722842261904762,
  'T_11': 3.1100852272727275,
  'T_15': 4.282366071428571,
  'T_16': 3.765625,
  'T_17': 3.453125,
  'T_3': 3.4225852272727275,
  'T_4': 3.7830528846153846,
  'T_7': 4.056175595238095,
  'T_8': 4.013822115384615}]


## The new DNC algorithm

To do: split into multiple functions that can be tested individually.

In [215]:
def dnc_tree_2(dm, taxa, base_case_size=5, core_size=5):
    if len(taxa) <= base_case_size:
        return dnc_neighborjoining(dm, taxa)
    else:
        core_taxa = random.sample(taxa, core_size) if len(taxa) > core_size else taxa
        non_core_taxa = set(taxa) - set(core_taxa)
        core_tree = dnc_neighborjoining(dm, core_taxa)
        center_vertex = get_center_vertex(core_tree)

        clades = get_clades_with_taxa_dists_to_center(core_tree, dm, center_vertex)

        # Map each taxon in core to its clade.
        # Also set the inferred distance from each taxon in core to the center.
        taxon_to_clade = {}
        for i in (0, 1, 2):
            for taxon, dist in clades[i].items():
                taxon_to_clade[taxon] = i
                dm.set(taxon, center_vertex, dist)

        # Assign non-core taxa to clades.

        # Map w -> sum_{x in C} d(w, x)
        sum_dist_w_to_core = {
            w: sum(dm.get(w, x) for x in core_taxa) for w in core_taxa
        }

        # Map (v, i) -> sum_{x in C_i} d(v, x)
        sum_dist_v_clade = {}
        for i in (0, 1, 2):
            for v in non_core_taxa:
                sum_dist_v_clade[(v, i)] = sum(dm.get(v, x) for x in clades[i])

        k = len(core_taxa)
        for v in non_core_taxa:
            smallest_q_so_far = None
            for w in core_taxa:
                q = (
                    (k - 2) * dm.get(v, w)
                    - sum_dist_v_clade[(v, 0)]
                    - sum_dist_v_clade[(v, 1)]
                    - sum_dist_v_clade[(v, 2)]
                    - sum_dist_w_to_core[w]
                )
                if smallest_q_so_far is None or q < smallest_q_so_far:
                    smallest_q_so_far = q
                    taxon_to_clade[v] = taxon_to_clade[w]  # w is v's new mate.

        # Esimate distance between each non-core taxon and the center vertex.

        for v in non_core_taxa:
            # Indices a and b are the two clades that v is not in.
            a, b = {0: (1, 2), 1: (0, 2), 2: (0, 1)}[taxon_to_clade[v]]
            n_taxa_in_a_and_b = len(clades[a]) + len(clades[b])
            # Estimate distance from v to center.
            sum_dist_estimates = (
                sum(dm.get(v, w) for w in clades[a])
                + sum(dm.get(v, w) for w in clades[b])
                - sum(clades[a].values())  # Could be calculated once and stored.
                - sum(clades[b].values())
            )
            # The distance is the average of the estimates.
            dm.set(v, center_vertex, sum_dist_estimates / n_taxa_in_a_and_b)

        # Finally ensure that the distance between center vertex and iteself is set.
        dm.set(center_vertex, center_vertex, 0)

        subtaxa_sets = [[center_vertex], [center_vertex], [center_vertex]]
        for v in taxa:
            subtaxa_sets[taxon_to_clade[v]].append(v)

        # Recurse and get three subtrees, each sharing the center vertex.

        subtrees = []
        for subtaxa in subtaxa_sets:
            subtrees.append(
                dnc_tree_2(
                    dm,
                    subtaxa,
                    base_case_size=base_case_size,
                    core_size=core_size,
                )
            )

    # Return the subtrees merged on the center vertex.
    return connect_the_trees(subtrees, center_vertex)

## Testing

In [220]:
msa = MockMSA(n_taxa=30, taxa_length=10, alphabet="ab")

dm = partialdistancematrix.PartialDistanceMatrix(msa, "Distance function dummy")
t_dnc2 = dnc_tree_2(dm, msa.taxa(), base_case_size=3, core_size=5)

dm = partialdistancematrix.PartialDistanceMatrix(msa, "Distance function dummy")
t_nj = dnc_neighborjoining(dm, msa.taxa())

In [219]:
write_graph_data(t, dm)

Number of lines: 74


In [178]:
t = connect_the_trees([t1, t2, t3], "#7")

In [179]:
write_graph_data(t, dm)

Number of lines: 22


In [155]:
t = dnc_tree_2(dm, sub_taxa_sets[0], base_case_size=5, core_size=5)

In [150]:
t

[['#7', 'T_6'], ['#7', 'T_3', 'T_8'], ['#7', '#2', 'T_0', 'T_2']]

In [148]:
t.adjacencies

{'#2': ['#3'],
 '#3': ['#2', 'T_7', '#5'],
 'T_7': ['#3'],
 'T_1': ['#4'],
 '#4': ['T_1', 'T_4', '#5'],
 'T_4': ['#4'],
 'T_5': ['#5'],
 '#5': ['T_5', '#3', '#4']}

In [108]:
t.adjacencies

{'T_2': ['#6'],
 '#6': ['T_2', 'T_9', '#7'],
 'T_9': ['#6'],
 '#2': ['#7'],
 '#7': ['#2', '#6', '#8'],
 'T_0': ['#8'],
 '#8': ['T_0', 'T_1', '#7'],
 'T_1': ['#8']}

In [65]:
print(t.adjacencies)

{'#2': ['#4'], '#4': ['#2', 'T_2', 'T_9'], 'T_2': ['#4'], 'T_9': ['#4']}


In [49]:
t.adjacencies

{'#2': ['T_9'], 'T_9': ['#2']}

In [25]:
class TestMSA:
    def __init__(self):
        self._taxa = [str(i) for i in range(1,12)]

    def can_retrieve_distances(self) -> bool:
        return True

    def distance(self, taxon1, taxon2):
        return float(int(taxon1) * int(taxon2))

    def taxa(self):
        return self._taxa

    def distance_matrix(self):
        """
        Not needed but for visualisation and debugging purposes.
        """
        df = pd.DataFrame(index=self.taxa(), columns=self.taxa())
        for taxon1, taxon2 in product(self.taxa(), self.taxa()):
            df.loc[taxon1, taxon2] = self.distance(taxon1, taxon2)
        return df

In [26]:
msa = TestMSA()
dm = partialdistancematrix.PartialDistanceMatrix(msa, "Distance function dummy")

core_taxa = ["1", "2", "3", "4", "5", "6", "7", "8"]
non_core_taxa = ["9", "10", "11"]

t = Tree()
edges = """
1 a 5
2 a 6
a b 7
8 b 5
b c 2
c d 1
d 3 3
d 4 2
c e 8
e 7 9
e f 5
f 5 4
f 6 3
"""

for line in edges.strip().split("\n"):
    x, y, weight = line.split()
    t.add_edge(x, y)
    if x not in dm._dm:
        dm._dm[x] = {}
    if y not in dm._dm:
        dm._dm[y] = {}
    dm.set(x, y, float(weight))

In [27]:
t = dnc_tree_2(dm, msa.taxa(), base_case_size=5, core_size=5)

[['#1', '5', '9'],
 ['#1', '1', '3', '4', '6', '7', '10', '11'],
 ['#1', '2', '8']]
----
{'1': 1,
 '10': 1,
 '11': 1,
 '2': 2,
 '3': 1,
 '4': 1,
 '5': 0,
 '6': 1,
 '7': 1,
 '8': 2,
 '9': 0}


In [243]:
t = dnc_neighborjoining(dm, core_taxa)


Number of lines: 14
Center vertex: #5
{'T_12': 2.75, '#3': 2.75, '#4': 2.25}


In [290]:
write_graph_data(t, dm)
center = get_center_vertex(t)
print(f"Center vertex: {center}")

Number of lines: 26
Center vertex: c


In [347]:
clades = get_clades_with_taxa_dists_to_center(t, dm, center)

# Map each taxon in core to its clade.
# Also set the inferred distance from each taxon in core to the center.
taxon_to_clade = {}
for i in (0, 1, 2):
    for taxon, dist in clades[i].items():
        taxon_to_clade[taxon] = i
        dm.set(taxon, center, dist)

pprint(clades)

[{'1': 14.0, '2': 15.0, '8': 7.0},
 {'3': 4.0, '4': 3.0},
 {'5': 17.0, '6': 16.0, '7': 17.0}]


$$Q(v, w) = (n - 2) d(v, w) - \sum_{x=1}^n d(v, x) - \sum_{x=1}^n d(w, x)$$


$C = \text{core vertices}$, $|C| = k$,

$A = C \cup \{v\}$, $|A| = n = k + 1$

$$Q(v, w) = (k + 1 - 2) d(v, w) - \sum_{x \in A} d(v, x) - \sum_{x \in A} d(w, x)$$

$$Q(v, w) = (k - 1) d(v, w) - \sum_{x \in C} d(v, x) - d(v,v) - \sum_{x \in C} d(w, x) - d(w, v)$$

$$Q(v, w) = (k - 2) d(v, w) - \sum_{x \in C} d(v, x) - \sum_{x \in C} d(w, x)$$

$$Q(v, w) = (k - 2) d(v, w) - \sum_{x \in C_0} d(v, x) - \sum_{x \in C_1} d(v, x) - \sum_{x \in C_2} d(v, x) - \sum_{x \in C} d(w, x)$$

where $C_0, C_1, C_2$ are the taxa from each clade, respectively.

In [292]:
# Map w -> sum_{x in C} d(w, x)
sum_dist_w = {w: sum(dm.get(w, x) for x in core_taxa) for w in core_taxa}

# Map (v, i) -> sum_{x in C_i} d(v, x)
sum_dist_v_clade = {}
for i in (0, 1, 2):
    for v in non_core_taxa:
        sum_dist_v_clade[(v, i)] = sum(dm.get(v, x) for x in clades[i])

k = len(core_taxa)
for v in non_core_taxa:
    smallest_q_so_far = None
    for w in core_taxa:
        q = (
            (k - 2) * dm.get(v, w)
            - sum_dist_v_clade[(v, 0)]
            - sum_dist_v_clade[(v, 1)]
            - sum_dist_v_clade[(v, 2)]
            - sum_dist_w[w]
        )
        if smallest_q_so_far is None or q < smallest_q_so_far:
            smallest_q_so_far = q
            taxon_to_clade[v] = taxon_to_clade[w]  # w is v's new mate.

In [350]:
taxon_to_clade["9"] = 2
taxon_to_clade["10"] = 1
taxon_to_clade["11"] = 0

print(taxon_to_clade)

{'1': 0, '2': 0, '8': 0, '3': 1, '4': 1, '7': 2, '5': 2, '6': 2, '9': 2, '10': 1, '11': 0}


clades is a list.

first element of clades is a dict representing clade 0.

This dict maps taxon x in clade 0 to the path weight from x to center vetrex.



In [357]:
# Esimate distance each v in non-core taxa to the center vertex.

for v in non_core_taxa:
    # Indices a and b are the two clades that v is not in.
    a, b = {0: (1, 2), 1: (0, 2), 2: (0, 1)}[taxon_to_clade[v]]
    n_taxa_in_a_and_b = len(clades[a]) + len(clades[b])
    # Estimate distance from v to center.
    sum_dist_estimates = (
        sum(dm.get(v, w) for w in clades[a])
        + sum(dm.get(v, w) for w in clades[b])
        - sum(clades[a].values())  # Can be calculated once and stored.
        - sum(clades[b].values())
    )
    # The distance is the average of the estimates.
    dm.set(v, center, sum_dist_estimates / n_taxa_in_a_and_b)

In [300]:
(9*(8+1+2)-sum(clades[0].values())+9*(3+4)-sum(clades[1].values())) / 5

23.8

In [302]:
dm.get("11", "c")

43.6

In [387]:
dm.get("11", "c")

43.6

# Old stuff

## Plot tree to visualise center vertex calculations

In [378]:
msa = MockMSA(n_taxa=10, taxa_length=10, alphabet="ab")
dm = partialdistancematrix.PartialDistanceMatrix(msa, "Distance function dummy")
t = dnc_neighborjoining(dm, msa.taxa())

c = get_center_vertex(t)

In [386]:
def write_tree(t):
    lines = []
    for v, neighbors in t.adjacencies.items():
        for n in neighbors:
            lines.append(f"{v} {n}")

    text = "\n".join(lines)
    (Path.home() / "temp" / "dnc_neighborjoining.txt").write_text(text)

In [ ]:
def write_tree_with_center_vertex(t):
    lines = []
    for v, neighbors in t.adjacencies.items():
        for n in neighbors:
            v = "Center" if v == c else v
            n = "Center" if n == c else n
            lines.append(f"{v} {n}")

    text = "\n".join(lines)
    (Path.home() / "temp" / "dnc_neighborjoining.txt").write_text(text)

In [365]:
# Create dict mapping letter a-f to a random integer in [0, 100).
d = {chr(i): random.randint(0, 4) for i in range(ord("a"), ord("f") + 1)}
d

{'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 1}

In [367]:
# return key in d with the smallest value.
min(d, key=d.get)

'a'

In [342]:
vertex_weights, edge_weights = get_weights(t)

In [351]:
plot_edges = {}
for x, y in edge_weights:
    if (x, y) in plot_edges or (y, x) in plot_edges:
        pass
    else:
        weight = f"{edge_weights[(x, y)]}/{edge_weights[(y, x)]}"
        plot_edges[(x, y)] = weight

In [352]:
for (x, y), weight in plot_edges.items():
    print(f"{x}/{vertex_weights[x]} {y}/{vertex_weights[y]} {weight}")

T_8/7 #0/6 1/7
T_2/6 #5/5 1/6
T_9/5 #4/4 1/5
T_3/7 #1/6 1/7
T_6/7 #1/6 1/7
#1/6 #3/5 2/6
T_4/7 #2/6 1/7
T_5/7 #2/6 1/7
#2/6 #3/5 2/6
#3/5 #4/4 3/5
#4/4 #7/4 4/4
T_0/6 #6/5 1/6
T_1/6 #6/5 1/6
#6/5 #7/4 2/5
#7/4 #5/5 5/3
#5/5 #0/6 6/2
#0/6 T_7/7 7/1


# DNC tree

In [17]:
msa = MockMSA(n_taxa=10, taxa_length=10, alphabet="ab")
dm = partialdistancematrix.PartialDistanceMatrix(msa, "Distance function dummy")

t = init_module.dnc_tree(
    dm,
    msa.taxa(),
    max_n_attempts=100,
    max_clade_size=0.5,
    base_case_size=5,
    first_triple=None,
    verbose=[],
)

	d(#0, T_3) = 2.8
	d(#0, T_7) = 1.83
	d(#0, T_6) = 2.67
	d(#1, T_9) = 1.15
	d(#1, T_2) = 1.83
	d(#1, #0) = -0.02


In [18]:
write_graph_data(t, dm)

Number of lines: 34
